In [ ]:
#library
import pandas as pd
import seaborn as sns
import numpy as np
import math as math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import os
import plotly.graph_objects as go
import plotly.figure_factory as ff
import upsetplot
import umap
from Bio import SeqIO
from Levenshtein import distance
from matplotlib_venn import venn3
import re
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
pd.set_option('max_colwidth',260)
from sklearn.model_selection import train_test_split
from sklearn import metrics
import itertools
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from scipy.stats import pearsonr
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import plotly.subplots as sp
from sklearn.metrics import r2_score
from scipy import stats
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import cross_val_predict
from scipy.optimize import minimize
from sklearn.linear_model import Ridge
from scipy.stats import spearmanr



In [ ]:
#load data
# promoter ~ motif syntax feature
df_syntax = pd.read_csv("./df_syntax.csv",sep=',',header=0,index_col=0) #include all var
df_syntax

#RD model
X = pd.concat([df_syntax["GC_CONTENT_normalized"],df_syntax.iloc[:, 11:]],axis=1)  #所有系数
y = df_syntax['log2B']

#  PCC 
def pcc_score(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

pcc_scorer = make_scorer(pcc_score)

# create Ridge model
ridge = Ridge()

# 定义要搜索的参数空间
param_grid = {
    'alpha': [6],
    'fit_intercept': [True],
    'solver': ['auto']
}

# create GridSearchCV 
grid_search = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    scoring=pcc_scorer,  #  PCC 
    n_jobs=-1,
    cv=10,
    verbose=3
)

# fit
grid_search.fit(X, y)

# best_params
best_params = grid_search.best_params_
print("Best parameters: ", best_params)



# best_params >  Ridge 
best_ridge = Ridge(**best_params)

# define KFold CV
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

#  cross_val_predict 
y_cv_pred = cross_val_predict(best_ridge, X, y, cv=kfold)

# Calculate R2, PCC, and SCC
r2 = metrics.r2_score(y, y_cv_pred)
pcc = stats.pearsonr(y, y_cv_pred)[0]
scc = stats.spearmanr(y, y_cv_pred)[0]
mse = metrics.mean_squared_error(y, y_cv_pred)
# Print scores
print("R2 score:", r2)
print("PCC score:", pcc)
print("SCC score:", scc)
print("MSE score:", mse)
print("PCC p-value:", p_value)



In [ ]:
# feature importance

coefs = []

for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    best_ridge = Ridge(**best_params)

    best_ridge.fit(X_train, y_train)

    coefs.append(best_ridge.coef_)

coefs = np.array(coefs)

coefs_df = pd.DataFrame(coefs, columns=X.columns)

print(coefs_df)
coefs_df